DSC160 Data Science and the Arts - Twomey - Spring 2020 - [dsc160.roberttwomey.com](http://dsc160.roberttwomey.com)

# Advanced Image Features

There are a variety of advanced image features we can use to develop more nuanced interpretations of images. This notebook demonstrates basic edge calculation, a Hough line transform, and pixel-wise entropy within the image.

This notebook isn't exhaustive: I'd suggest you explore [opencv](https://opencv.org/), [scikit-image](https://scikit-image.org/), and other image processing libraries to discover additional possbilities for an image-based cultural analysis.

## Calculating Edges Within an Image

There are a variety of ways to calculate edges within an image. The resulting edge image can be the basis for contour detection or other advanced processes, or can be used to directly estimate the number of pixels (amount) of high spatial frequency transitions within the image. 

### Soble Operator for Edge Detection

The Sobel operator performs a 2-D spatial gradient measurement on an image and so emphasizes regions of high spatial frequency that correspond to edges. 

The sobel filter uses two 3 x 3 kernels. One for changes in the horizontal direction, and one for changes in the vertical direction. The two kernels are convolved with the original image to calculate the approximations of the derivatives

If we define Gx and Gy as two images that contain the horizontal and vertical derivative approximations respectively, the computations are:

${ G }_{ x }=\left[ \begin{matrix} 1 & 0 & -1 \\ 2 & 0 & -2 \\ 1 & 0 & -1 \end{matrix} \right] *Image $

${ G }_{ y }=\left[ \begin{matrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{matrix} \right] *Image $

At each pixel in the image, the gradient approximations given by ${ G }_{ x }$ and ${ G }_{ y }$ are combined to give the gradient magnitude, using $\sqrt { { { G }_{ x } }^{ 2 }+{ { G }_{ y } }^{ 2 } } $

The gradient’s direction is calculated using: $\Theta =arctan\left( \frac { { G }_{ x } }{ { G }_{ y } }  \right) $

### Calculate edges within the value channel of an image

In [ ]:
from scipy import ndimage
import scipy.misc
from skimage import data, io
from skimage.color import rgb2hsv

import matplotlib.pyplot as plt
import numpy as np

First we load a test image and convert it to hsv

In [ ]:
face = scipy.misc.face()
rgb_img = face
hsv_img = rgb2hsv(rgb_img)

Extract the H, S, and V channels

In [ ]:
hue_img = hsv_img[:, :, 0]
saturation_img = hsv_img[:,:, 1]
value_img = hsv_img[:, :, 2]

Display the H, S, V images:

In [ ]:
fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(16, 8))

ax0.imshow(rgb_img)
ax0.set_title("RGB image")
ax0.axis('off')
ax1.imshow(hue_img, cmap='hsv')
ax1.set_title("Hue channel (H)")
ax1.axis('off')
ax2.imshow(saturation_img)
ax2.set_title("Saturation channel (S)")
ax2.axis('off')
ax3.imshow(value_img)
ax3.set_title("Value channel (V)")
ax3.axis('off')

fig.tight_layout()

Use the ndimage (multi-dimensional image) sobel filter method:

In [ ]:
sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
edge_image = np.hypot(sobel_x, sobel_y)

ax = plt.imshow(edge_image, cmap='gray')

Note that the edges correspond to the areas of sharp contrast/pixel-pixel transition within the value image above.

### Calculate Edges on a painting

In [ ]:
painting = io.imread("https://images.rkd.nl/rkd/thumb/650x650/985b50b4-1378-78dd-52ce-1eee52771b4f.jpg")
ax = plt.imshow(painting)

Convert the image to HSV and extract the value channel

In [ ]:
hsv_img = rgb2hsv(painting)
value_img = hsv_img[:, :, 2]

In [ ]:
ax = plt.imshow(value_img, cmap='gray')

Calculate the x and y component of the sobel:

In [ ]:
sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
edge_image = np.hypot(sobel_x, sobel_y)

Display the resulting edge image

In [ ]:
ax = plt.imshow(edge_image, cmap='gray')

NOTE: the resulting edge image shows high intensity where hard edges are found within an image, and low intensity in areas of uniform color. 

The sum of the edge image (normalized by pixel count) is an approximation of how many edges are found within the image

In [ ]:
print("sum of edge image:", np.sum(edge_image)/ edge_image.size)

## OpenCV Implementations of Techniques

OpenCV (computer vision) has lots of great advanced image processing techniques: https://opencv.org/

Depending on your working platform, we may need to install opencv. The following pip command will install opencv and oencv-contrib (additional methods) for the current user on datahub. Uncomment (remove the `#`) and then run the cell. (FYI the `!` is jupyter magic to run a bash command, as if you opened a terminal and executed the following text)

In [ ]:
#!pip install opencv-python-headless --user
#!pip install opencv-contrib-python-headless --user

Import modules and do some setup

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
from scipy import ndimage
from skimage.color import rgb2hsv
import skimage

### Using OpenCV to find features in two paintings by Piet Mondrian

Load two images of paintings from the artist Piet Mondrian, a prototypical example of an artist who over the course of his career progressed from a realistic/representational style to pure geometric abstraction. See his complete works here: [http://pietmondrian.rkdmonographs.nl/](http://pietmondrian.rkdmonographs.nl/).

We will load in one abstract painting

In [ ]:
abstract = io.imread("https://images.rkd.nl/rkd/thumb/650x650/985b50b4-1378-78dd-52ce-1eee52771b4f.jpg")
ax = io.imshow(abstract)

and one landscape image

In [ ]:
landscape = io.imread('https://images.rkd.nl/rkd/thumb/650x650/bcb9558d-08a1-a57f-b5fc-ec562c446838.jpg')
ax = io.imshow(landscape)

### Calculate edge images for each painting

In [ ]:
# grab value channel
hsv_img = rgb2hsv(landscape)
value_img = hsv_img[:, :, 2]

# calculate sobel in x and y directions
sobel_x = ndimage.sobel(value_img, axis=1, mode='constant')
sobel_y = ndimage.sobel(value_img, axis=0, mode='constant')
landscape_edge_image = np.hypot(sobel_x, sobel_y)

# display figure
fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(16, 4))

ax0.imshow(landscape)
ax0.set_title("Painting")
ax0.axis('off')
ax1.imshow(landscape_edge_image, cmap='gray')
ax1.set_title("Edge (Magnitude of X-Y Sobel)")
ax1.axis('off')
ax2.imshow(sobel_x, cmap='gray')
ax2.set_title("X Sobel")
ax2.axis('off')
ax3.imshow(sobel_y, cmap='gray')
ax3.set_title("Y Sobel")
ax3.axis('off')
fig.tight_layout()

# print stats
print("mean of edge_image (edginess):", landscape_edge_image.mean())

In [ ]:
# grab value channel
hsv_img = rgb2hsv(abstract)
value_img = hsv_img[:, :, 2]

# calculate sobel in x and y directions
sobel_x = ndimage.sobel(value_img, axis=1, mode='constant')
sobel_y = ndimage.sobel(value_img, axis=0, mode='constant')
abstract_edge_image = np.hypot(sobel_x, sobel_y)

# display figure
fig, (ax0, ax1, ax2, ax3) = plt.subplots(ncols=4, figsize=(16, 4))

ax0.imshow(abstract)
ax0.set_title("Painting")
ax0.axis('off')
ax1.imshow(sobel_x, cmap='gray')
ax1.set_title("X Sobel")
ax1.axis('off')
ax2.imshow(sobel_y, cmap='gray')
ax2.set_title("Y Sobel")
ax2.axis('off')
ax3.imshow(abstract_edge_image, cmap='gray')
ax3.set_title("Edge Image (Magnitude of X,Y Sobel)")
ax3.axis('off')
fig.tight_layout()

# print stats
print("mean of edge_image (edginess):", abstract_edge_image.mean())

## Calculate Hough Lines

Hough transform is a feature extraction method for detecting simple shapes such as circles, lines etc in an image.

It involves:
- representing a shape in mathematical form. For example, a line can be represented by two parameters (slope, intercept) and a circle has three parameters — the coordinates of the center and the radius (x, y, r).
- building a histogram using the parameters used to represent the shape
- thresholding the histogram to output presence of shapes

For example, consider hough lines.

We know the polar form of a line is represented as:
$\rho =x\cos { \theta  } +y\sin { \theta  } $

Here $\rho$ represents the perpendicular distance of the line from the origin in pixels, and $\theta$ is the angle measured in radians.
When we say that a line in 2D space is parameterized by $\rho$ and $\theta$, it means that if we any pick a ($\rho$, $\theta$), it corresponds to a line.

We build an accumulator matrix, a histogram matrix with each cell denoting number of pixels parametrized by same $\rho$ and $\theta$

We can simply select the bins in the accumulator above a certain threshold to find the lines in the image. If the threshold is higher, you will find fewer strong lines, and if it is lower, you will find a large number of lines including some weak ones.

In [ ]:
# convert from skimage to opencv image format
from skimage import exposure
temp = exposure.rescale_intensity(abstract_edge_image, out_range=(-1.0, 1.0))
edges = skimage.img_as_ubyte(np.clip(temp, -1, 1))

# calculate hough lines
lines = cv2.HoughLines(edges,1,np.pi/180,100)
result = np.copy(abstract)
for x in range(0, len(lines)):    
    for rho, theta in lines[x]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv2.line(result,(x1,y1),(x2,y2),(255,255,0),2)

# show results
ax = plt.imshow(result[:,:,::])
# cv2.imwrite('houghlines3.jpg',img)

How many horizontal/vertical lines did we find?

In [ ]:
print(len(lines))

Extension: try this again with the landscape image above, and see if we detect any horizontal or vertical lines.

Further reading on the Hough Transform on [opencvy python tutorials](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html)

### Probabilistic Hough

Probabilistic Hough Transform is an optimization of Hough Transform. It doesn’t take all the input points into consideration, instead take only a random subset of points and that is sufficient for shape detection.

One of the easiest probabilstic method is to choose m input points from the set M input points. The complexity of the voting stage reduces from $O(M.{ N }_{ \Theta  })$ to $O(m.{ N }_{ \Theta  })$. This works because a random subset of M will fairly represent the edge points and the surrounding noise and distrotion.

Smaller value of m will result in fast computation but lower accuracy. So the value of m should be appropriately chosen with respect to M.

In [ ]:
temp = exposure.rescale_intensity(abstract_edge_image, out_range=(-1.0, 1.0))

edges = skimage.img_as_ubyte(np.clip(temp, -1, 1))

# Probabilistic Hough Transform
minLineLength = 400
maxLineGap = 10

lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
result = abstract.copy()

for x in range(0, len(lines)):    
    for x1,y1,x2,y2 in lines[x]:
        cv2.line(result,(x1,y1),(x2,y2),(0,255,255),5)

# cv2.imwrite('houghlines5.jpg',edges)
ax = plt.imshow(result[:,:,::])

In [ ]:
print(len(lines))

## Image entropy

The entropy H of an image is defined as

$H=-\sum _{ k=0 }^{ M-1 }{ { p }_{ k }\log _{ 2 }{ ({ p }_{ k }) }  } $

Entropy can be considered as the spread of states. A low entropy system occupies a small number of such states, while a high entropy system occupies a large number of states.

For example, in an 8-bit pixel there are 256 such states. If all such states are equally occupied, as they are in the case of an image which has been perfectly histogram equalized, the spread of states is a maximum, as is the entropy of the image. On the other hand, if the image has been thresholded, so that only two states are occupied, the entropy is low. If all of the pixels have the same value, the entropy of the image is zero.

The example below iterates across all positions in the image, calculating the entropy at each point based on the surrounding values within a selected region (in this case a disk of radius 10). In the output image (entropy image) each pixel corresponds to the calculated entropy score for the surrounding values in the original input image.

More information on image entropy: https://scikit-image.org/docs/dev/auto_examples/filters/plot_entropy.html

In [ ]:
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2gray

Entropy calculation using builtin function.

In [ ]:
img = landscape
gray_img = rgb2gray(img)
entr_img = entropy(gray_img, disk(10))

ax = io.imshow(entr_img)

Notice: `disc(10)` above is the structuring element, a disc of radius 10. Entropy is calculated for each pixel based on the values within a circle of radius 10 around that point. There are other choices of shape and obviously radius/size, which will affect the output.

In [ ]:
ax = io.imshow(img)

In [ ]:
img = abstract
gray_img = rgb2gray(img)
entr_img = entropy(gray_img, disk(10))

ax = io.imshow(entr_img)

In [ ]:
ax = io.imshow(img)

## Image Energy


More information on Dual Gradient image energy (in the context of Seam Carving): https://www.cs.princeton.edu/courses/archive/spring13/cos226/assignments/seamCarving.html

In [ ]:
def calcDGenergy(img):
    # from from https://stackoverflow.com/a/48974892

    #convert from uint8 to int64 to prevent overflow problems
    arr = np.array(img, dtype = int)

    #calculate squared difference ((x-1, y) - (x+1, y))^2 for each R, G and B pixel
    deltaX2 = np.square(np.roll(arr, -1, axis = 0) - np.roll(arr, 1, axis = 0))

    #same for y axis
    deltaY2 = np.square(np.roll(arr, -1, axis = 1) - np.roll(arr, 1, axis = 1))

    #add R, G and B values for each pixel, then add x- and y-shifted values
    dualEnergy = np.sum(deltaX2, axis = 2) + np.sum(deltaY2, axis = 2)
    return dualEnergy

In [ ]:
dgEnergy = calcDGenergy(abstract)
plt.imshow(dgEnergy)
plt.show()

print("min: ", dgEnergy.min())
print("max: ", dgEnergy.max())
print("mean: ", dgEnergy.mean())

In [ ]:
dgEnergy = calcDGenergy(landscape)
plt.imshow(dgEnergy)
plt.show()

print("min: ", dgEnergy.min())
print("max: ", dgEnergy.max())
print("mean: ", dgEnergy.mean())

## Further Directions

As mentioned above, there are many further techniques that can be used to extract features from and study/describe paintings.

If you have any experience with this, share your ideas with the class on Piazza!